In [2]:
# Load the aiida profile
from aiida import load_profile
aiida_profile = load_profile()
print(f'profil name: {aiida_profile.name}')

## Loading the some require packages and module
from aiida_kkr.workflows import ( kkr_flex_wc, kkr_bs_wc,
                                 kkr_imp_wc, kkr_imp_sub_wc
                                )
from aiida_kkr.calculations import KkrCalculation, KkrimpCalculation
from aiida.orm import (Group, load_group, load_node, Dict, Code, QueryBuilder, WorkChainNode
                       , CalcJobNode, StructureData, KpointsData)
from aiida.transports import transport
from aiida_kkr.tools import kkrparams, find_parent_structure, kkrparams
from aiida.engine import submit

from aiida.tools.data.array.kpoints import get_explicit_kpoints_path

profil name: default


In [3]:
import numpy as np

In [4]:
# One very frequently used option
metadata_option_1 = {'max_wallclock_seconds': 36000,
                     'resources': {'tot_num_mpiprocs': 48, 'num_machines': 1},
                     'custom_scheduler_commands': 
                     '#SBATCH --account=jara0191\n\nulimit -s unlimited; export OMP_STACKSIZE=2g; export OMP_NUM_THREADS=2',
                     'withmpi': True
                    }

# One very frequently used option
metadata_option_2 = {'max_wallclock_seconds': 36000,
                     'resources': {'tot_num_mpiprocs': 48, 'num_machines': 1},
              'custom_scheduler_commands': 
              '#SBATCH --account=jara0191\n\nulimit -s unlimited; export OMP_STACKSIZE=2g',
              'withmpi': True
                    }
oscar_matadata = {'max_wallclock_seconds': 8*60*60,
                  'resources':{'tot_num_mpiprocs': 12, 
                  'num_machines': 4},
                  'custom_scheduler_commands': '#SBATCH -p oscar --nodes=1 --ntasks=12\n\nulimit -s unlimited; export OMP_STACKSIZE=2g',
                  'withmpi':True
                 }

oscar_matadata = {'withmpi': True,
         'resources': {'num_machines': 1, 'tot_num_mpiprocs': 12},
         'queue_name': 'oscar',
         'max_wallclock_seconds': 86400}

voro_code = Code.get_from_string('voro@claix18_init')
kkr_code = Code.get_from_string('kkr@claix18_init')
kkrimp_code = Code.get_from_string('kkrflex@claix18_init')
iffslurm_voro = Code.get_from_string('voro@iffslurm')
iffslurm_kkr= Code.get_from_string('kkr@iffslurm')
iffslurm_kkrimp= Code.get_from_string('kkrflex@iffslurm')

In [6]:
#Some kpints collectionn:
# 1D kpoints k_x
k=[]
for i in np.arange(-0.1, 0.1, 0.2/500):
#    print(i)
    k.append([i,0,0])
kpt_x= KpointsData()
kpt_x.set_cell([[1,0,0], [0,1,0], [0,0,1]])
kpt_x_arr= np.array(k)

random_min = 1
index=0
for i in range(np.shape(kpt_x_arr)[0]):
    abs_val= np.sqrt(sum([i**2 for i in kpt_x_arr[i,:]]))
    if random_min>abs_val:
        random_min= abs_val
        index= i
        vec= kpt_x_arr[i,:]
kpt_x.set_kpoints(k, labels=[(index,'GAMMA')])


In [80]:
## To extract the host calcualtion data for structure node 234
Bs_group= load_group(122)
finished_labels= [node.label for node in Bs_group.nodes]
for j in np.linspace(0.0,1,1):
    print(j)
    kkr_imp_wc= load_node(16100)
    # initial remote data host from the converged host Bi_2Te_3 calculation
    RemoteDataHost= kkr_imp_wc.get_incoming(link_label_filter='remote_data_host').all()[0].node 
    # initial converged calc 
    calc= RemoteDataHost.get_incoming().all()[0].node
    struc= find_parent_structure(calc)
    #struc= struc.cell
    wf_parameters={
        'EMIN':-1.5,
        'EMAX': 1.5,
        'NPT2': 192,
        '<SOCSCL>': [j for i in range(10)]
    }
    #kpts= get_explicit_kpoints_path(struc).get('explicit_kpoints') # 101056
    kpts= kpt_x # del(101074, 101084, 101111, 101118, 101130, 101142, 101174, 101192, 101202, 101212)

    # final bs for Bi_2Te_3: 101249
    label= 'SOCSCL:'+str("%.2f" % j)
    bs_builder= kkr_bs_wc.get_builder()
    bs_builder.wf_parameters= Dict(dict=wf_parameters)
    bs_builder.kkr= kkr_code
    bs_builder.remote_data= RemoteDataHost
    bs_builder.options= Dict(dict=metadata_option_1)
    bs_builder.kpoints= kpts
    bs_builder.metadata.label= label
    if label in finished_labels:
        print(f'label: {label} is already exist in group 122')
        continue
    print(f'label: {label} is not found in the group 122 and the bs of the same is begin launched')
#    bs_calc= submit(bs_builder)
    Bs_group.add_nodes(bs_calc)

    


0.0
label: SOCSCL:0.00 is not found in the group 122 and the bs of the same is begin launched


In [78]:
bs_calc

<WorkChainNode: uuid: f7195d9b-8f2d-4676-b712-88cffe6ef5c1 (pk: 102160) (aiida.workflows:kkr.bs)>

In [52]:
bs_group= load_group(122)
nodes= list(bs_group.nodes)[0:5]
#plot_kkr(nodes)


In [82]:
bs_group= load_group(122)
nodes= list(bs_group.nodes)[-2:]

from aiida_kkr.tools import plot_kkr
from matplotlib import pyplot as plt
import matplotlib.gridspec as gdspec
plt.figure(num= 'bs:Bi_2Te_3', figsize=(30,20))
i_num=len(nodes)
j_num=5
#gdspec.GridSpec(i_num,j_num)
for i in range(i_num):
        print(nodes[i].inputs.wf_parameters.get_dict()['<SOCSCL>'])
        #plot_kkr(nodes[i], silent=True)
        #plt.show()
#bs= load_node(load_group(122))
#plot_kkr(bs)

[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]


<Figure size 2160x1440 with 0 Axes>

In [45]:
bs_group= load_group(122)
nodes= bs_group.nodes
for node in nodes:
#    print(node.process_state)
    if str(node.process_state)=='ProcessState.EXCEPTED':
        print('process status: ',node.process_status)
        print('node label', node.label)
        print('node pk : ', node.pk)
        print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')
        
        
 #   print(node.label)
#    node.label=label

In [26]:
node_101161= load_node(101554)
submit(node_101161.get_builder_restart())

#node_101161.inputs.parameters.get_dict()

<WorkChainNode: uuid: 5c5eccae-9db0-4c1a-bb2b-d655fdac1f38 (pk: 101799) (aiida.workflows:kkr.bs)>

In [121]:
#KpointsData(kpts)
kpts_arry= kpts.get_kpoints()
max_xyz= kpts_arry.max(axis=0)/4
max_r= np.sqrt(np.sum([i**2 for i in max_xyz[:]]))
max_r
trimed_kpts= []
for i in range(np.shape(kpts_arry)[0]):
    if np.sqrt(np.sum([j**2 for j in list(kpts_arry[i,:])])) <= max_r:
        trimed_kpts.append(list(kpts_arry[i,:]))

In [122]:
list(kpts_arry[5,:])

[0.22727272727272724, 0.2272727272727272, 0.22727272727272718]

In [66]:
node_10189= load_node(101908)
node_10189.inputs.wf_parameters.get_dict()

{'EMAX': 1.5,
 'EMIN': -1.5,
 'NPT2': 192,
 '<SOCSCL>': [0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7, 0.7]}